In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Polycystic_Kidney_Disease/GSE74451'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Identification of a novel risk factor for intracranial aneurysms in ADPKD using iPSC models [Agilent]"
!Series_summary	"Cardiovascular complications are the leading cause of death in autosomal dominant polycystic kidney disease (ADPKD), and intracranial aneurysm (ICA) causing subarachnoid hemorrhage is among the most serious complications. The diagnostic and therapeutic strategies for ICAs in ADPKD have not been fully established. We here generated induced pluripotent stem cells (iPSCs) from seven ADPKD patients, including four with ICAs. The vascular cells differentiated from ADPKD-iPSCs showed altered Ca2+ entry and gene expression profiles compared with those from control-iPSCs. We found that the expression level of a metalloenzyme gene, matrix metalloproteinase (MMP) 1, was specifically elevated in the iPSC-derived endothelia from ADPKD patients with ICAs. Furthermore, we confirmed a statistically significant correlation between the serum MMP1

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True  # As the dataset mentions analyzing gene expression profiles

# Identify the keys for relevant variables in the sample characteristics dictionary
sample_characteristics = {
    0: ['subject/sample source id: TIG114 4F1', 'subject/sample source id: TIG118 4F1', 'subject/sample source id: TIG119 4F1', 'subject/sample source id: TIG121 4F4', 'subject/sample source id: TIG975E4', 'subject/sample source id: 585A1', 'subject/sample source id: 585B1', 'subject/sample source id: TIG107 4F1', 'subject/sample source id: TIG120 4F1', 'subject/sample source id: TIG120 3F7', 'subject/sample source id: TIG975E2', 'subject/sample source id: CiRA00004', 'subject/sample source id: CiRA00005', 'subject/sample source id: CiRA00006', 'subject/sample source id: CiRA00007', 'subject/sample source id: CiRA00008', 'subject/sample source id: CiRA00009', 'subject/sample source id: CiRA00010'],
    1: ['subject status: healthy control', 'subject status: autosomal dominant polycystic kidney disease (ADPKD) patient'],
    2: ['gender: Male', 'gender: Female'],
    3: ['cell type: iPSC derived endothelial cells', 'cell type: iPSC derived vascular smooth muscle cells'],
    4: ['gender: Male', 'gender: Female']
}

# Assuming that age is not provided in this dataset based on available fields
trait_row = 1  # subject status
gender_row = 2  # gender

# Define the conversion functions for each variable
def convert_trait(value):
    value = value.split(':')[-1].strip().lower()
    if value == 'healthy control':
        return 0
    elif value == 'autosomal dominant polycystic kidney disease (adpkd) patient':
        return 1
    return None

def convert_age(value):
    return None

def convert_gender(value):
    value = value.split(':')[-1].strip().lower()
    if value == 'female':
        return 0
    elif value == 'male':
        return 1
    return None

# Save cohort information
save_cohort_info('GSE74451', './preprocessed/Polycystic_Kidney_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Polycystic_Kidney_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Polycystic_Kidney_Disease/trait_data/GSE74451.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1920920': [0, 1], 'GSM1920921': [0, 0], 'GSM1920922': [0, 1], 'GSM1920923': [0, 1], 'GSM1920924': [0, 0], 'GSM1920925': [0, 1], 'GSM1920926': [0, 1], 'GSM1920927': [0, 1], 'GSM1920928': [0, 0], 'GSM1920929': [0, 0], 'GSM1920930': [0, 0], 'GSM1920931': [0, 0], 'GSM1920932': [0, 1], 'GSM1920933': [0, 0], 'GSM1920934': [0, 0], 'GSM1920935': [0, 1], 'GSM1920936': [0, 1], 'GSM1920937': [1, 0], 'GSM1920938': [1, 0], 'GSM1920939': [1, 1], 'GSM1920940': [1, 1], 'GSM1920941': [1, 1], 'GSM1920942': [1, 0], 'GSM1920943': [1, 0], 'GSM1920944': [1, 0], 'GSM1920945': [1, 0], 'GSM1920946': [1, 1], 'GSM1920947': [1, 1], 'GSM1920948': [1, 1], 'GSM1920949': [1, 0], 'GSM1920950': [1, 0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315459', 'A_19_P00315482', 'A_19_P00315492', 'A_19_P00315493',
       'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315519'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'GB_ACC': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'LOCUSLINK_ID': [nan, nan, 50865.0, 23704.0, 128861.0], 'GENE_SYMBOL': [nan, nan, 'HEBP1', 'KCNE4', 'BPIFA3'], 'GENE_NAME': [nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4', 'BPI fold containing family A, member 3'], 'UNIGENE_ID': [nan, nan, 'Hs.642618', 'Hs.348522', 'Hs.360989'], 'ENSEMBL_ID': [nan, nan, 'ENST00000014930', 'ENST00000281830', 'ENST00000375454'], 'ACCESSION_STRING': [nan, nan, 'ref|NM_015987|ens|ENST00000014930|gb|AF117615|gb|BC016277', 'ref|NM_080671|ens|ENST00000281830|tc|THC2655788', 'ref|NM_178466|ens|ENST00000375454|ens|ENST00000471233|tc

### Step 6: Gene Identifier Mapping

In [7]:

# 1. Determine identifier and gene symbol keys based on previewed data
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and name the resulting dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Polycystic_Kidney_Disease/gene_data/GSE74451.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Polycystic_Kidney_Disease')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE74451', './preprocessed/Polycystic_Kidney_Disease/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Polycystic_Kidney_Disease/GSE74451.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Polycystic_Kidney_Disease', the least common label is '1.0' with 14 occurrences. This represents 45.16% of the dataset.
The distribution of the feature 'Polycystic_Kidney_Disease' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 15 occurrences. This represents 48.39% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

